# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [108]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [8]:
df1= pd.read_csv("LoanStats_2019Q1.csv")
df1.tail()

Notes offered by Prospectus (https://www.lendingclub.com/info/prospectus.action)
NaN                                              NaN 16000 16000 16000  36 months  16.14% 563.62 C   C4  Estimator/Supervisor 10+ years MORTGAGE 32000.0 Source Verified Jan-2019 Current n   NaN NaN debt_consolidation Debt consolidation 925xx CA  20.89 0   Dec-2010 0   35.0 117.0 13  1   17066.0 49.9% 15  f   14940.62 14940.62 1676.51 1676.51 1059.38 617.13  0.0 0.0 0.0 Apr-2019 563.62 May-2019 Apr-2019 0   NaN 1   Individual NaN NaN NaN 0   0   17066.0  1   0   0   0   55.0 0       NaN  3   3   2353  50.0 34200   0   0   1   3   1313    6446.0  60.7 0   0   55.0  96  1   1   0   1.0  35.0 12.0 35.0 0   7   8   8   8   1   13  14  8   13  0.0 0   0   3   93.3  37.5 1   0   34200    17066.0 16400 0       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN N   NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN N   NaN NaN NaN NaN NaN                                                NaN                              
                                                                        60 months  11.31% 350.36 B   B3  MATERIAL HANDLER     5 years   MORTGAGE 72000.0 Verified        Jan-2019 Current n   NaN NaN debt_consolidation Debt consolidation 850xx AZ  7.02  2   Sep-2005 0   8.0  64.0  12  1   11882   37.1% 39  w   15395.66 15395.66 1041.03 1041.03 604.34  436.69  0.0 0.0 0.0 Apr-2019 350.36 May-2019 Apr-2019 0   8.0 1   Individual NaN NaN NaN 0   0   225413.0 0   2   0   0   28.0 62953.0 79.0 0   0   5568  45.0 32000   2   2   4   1   18784.0 11705.0 49.1 2   0   159.0 129 25  24  3   25.0 NaN  1.0  9.0  2   4   5   5   9   20  9   16  5   12  0.0 0   2   0   94.3  0.0  1   0   251486.0 74835.0 23000 63090   NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN N   NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN N   NaN NaN NaN NaN NaN                                                NaN                              
                                                     29250 29250 29250  60 months  18.94% 757.8  D   D2  sr register csa      7 years   MORTGAGE 65000.0 Verified        Jan-2019 Current n   NaN NaN debt_consolidation Debt consolidation 774xx TX  29.52 0   Jan-2011 0   NaN  NaN   20  0   38465.0 69%   22  w   28347.49 28347.49 2211.84 2211.84 902.51  1309.33 0.0 0.0 0.0 Apr-2019 757.8  May-2019 Apr-2019 0   NaN 1   Individual NaN NaN NaN 0   0   204764   0   1   0   1   17.0 20323   79.0 1   3   13874 69.0 55505.0 0   0   0   4   10238   2985.0  87.0 0   0   78.0  95  7   7   1   19.0 NaN  13.0 NaN  0   13  18  13  13  3   18  18  18  20  NaN 0   0   1   100.0 76.9 0   0   237833.0 58788   38800 25728.0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN N   NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN N   NaN NaN NaN NaN NaN                                                NaN                              
Total amount funded in policy code 1: 1928448350 NaN NaN   NaN   NaN   NaN        NaN     NaN    NaN NaN NaN                  NaN       NaN      NaN     NaN             NaN      NaN     NaN NaN NaN NaN                NaN                NaN   NaN NaN   NaN NaN      NaN NaN  NaN   NaN NaN NaN     NaN   NaN NaN NaN      NaN      NaN     NaN     NaN     NaN     NaN NaN NaN NaN      NaN    NaN      NaN      NaN NaN NaN NaN        NaN NaN NaN NaN NaN NaN      NaN NaN NaN NaN NaN  NaN     NaN  NaN NaN NaN   NaN  NaN     NaN NaN NaN NaN NaN     NaN     NaN  NaN NaN NaN   NaN NaN NaN NaN NaN  NaN  NaN  NaN  NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN   NaN  NaN NaN NaN      NaN     NaN   NaN     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN                                                NaN                              
Total amount funded in policy code 2: 799382986  NaN NaN   NaN   NaN   NaN        NaN     NaN    NaN NaN NaN                  NaN       NaN      NaN     NaN             NaN      NaN     NaN NaN NaN NaN                NaN               

In [10]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()
df.tail()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
115670,5000.0,13.56%,169.83,MORTGAGE,48000.0,Not Verified,Jan-2019,Current,n,8.28,...,90.5,NaN,0.0,0.0,88613.0,9182.0,0.0,11413.0,N,N
115671,6000.0,6.46%,183.79,MORTGAGE,96000.0,Not Verified,Jan-2019,Current,n,0.31,...,100.0,0.0,1.0,0.0,33500.0,50.0,14900.0,0.0,N,N
115672,16000.0,16.14%,563.62,MORTGAGE,32000.0,Source Verified,Jan-2019,Current,n,20.89,...,93.3,37.5,1.0,0.0,34200.0,17066.0,16400.0,0.0,N,N
115673,16000.0,11.31%,350.36,MORTGAGE,72000.0,Verified,Jan-2019,Current,n,7.02,...,94.3,0.0,1.0,0.0,251486.0,74835.0,23000.0,63090.0,N,N
115674,29250.0,18.94%,757.80,MORTGAGE,65000.0,Verified,Jan-2019,Current,n,29.52,...,100.0,76.9,0.0,0.0,237833.0,58788.0,38800.0,25728.0,N,N


In [99]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head(5)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [100]:
df.dtypes[df. dtypes == 'object']


home_ownership          object
verification_status     object
issue_d                 object
loan_status             object
pymnt_plan              object
initial_list_status     object
next_pymnt_d            object
application_type        object
hardship_flag           object
debt_settlement_flag    object
dtype: object

In [110]:

# Create our features
X = pd.get_dummies(df,columns=["home_ownership","verification_status","issue_d","pymnt_plan","initial_list_status","next_pymnt_d","application_type","hardship_flag","debt_settlement_flag"])
X= X.drop(columns="loan_status")
X.describe()


# Create our target
y = df["loan_status"]
y

0        low_risk
1        low_risk
2        low_risk
3        low_risk
4        low_risk
           ...   
68812    low_risk
68813    low_risk
68814    low_risk
68815    low_risk
68816    low_risk
Name: loan_status, Length: 68817, dtype: object

In [102]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [103]:
y.count()

loan_status    68817
dtype: int64

In [114]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [111]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [117]:
Counter(y_train)

Counter({'low_risk': 51357, 'high_risk': 255})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [118]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 51357, 'high_risk': 51357})

In [119]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs',random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [122]:
y_pred = model.predict(X_test)

In [123]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6280192530976959

In [124]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   54,    38],
       [ 5663, 11450]], dtype=int64)

In [125]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.59      0.67      0.02      0.63      0.39        92
   low_risk       1.00      0.67      0.59      0.80      0.63      0.40     17113

avg / total       0.99      0.67      0.59      0.80      0.63      0.40     17205



### SMOTE Oversampling

In [126]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 51357, 'high_risk': 51357})

In [128]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model= LogisticRegression(solver='lbfgs',random_state=1)
model.fit(X_resampled, y_resampled)
y_pred= model.predict(X_test)

In [129]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6335702707578017

In [130]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   53,    39],
       [ 5287, 11826]], dtype=int64)

In [131]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.58      0.69      0.02      0.63      0.39        92
   low_risk       1.00      0.69      0.58      0.82      0.63      0.40     17113

avg / total       0.99      0.69      0.58      0.81      0.63      0.40     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [132]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 255, 'low_risk': 255})

In [136]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [137]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.49627603220536637

In [138]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  50,   42],
       [9428, 7685]], dtype=int64)

In [139]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.54      0.45      0.01      0.49      0.25        92
   low_risk       0.99      0.45      0.54      0.62      0.49      0.24     17113

avg / total       0.99      0.45      0.54      0.62      0.49      0.24     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [144]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'high_risk': 68458, 'low_risk': 62022})

In [145]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred=model.predict(X_test)

In [146]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6388040238923371

In [147]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  65,   27],
       [7340, 9773]], dtype=int64)

In [148]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.71      0.57      0.02      0.64      0.41        92
   low_risk       1.00      0.57      0.71      0.73      0.64      0.40     17113

avg / total       0.99      0.57      0.71      0.72      0.64      0.40     17205

